# Scenario

Your 5 year old brother has been using MS Word for a while, and is now curious about PowerPoint (the Orange Word, as he calls it). So you teach him the concept of splitting content into separate slides, each with their own pretty graphics and animations. He's fascinated by this.

> You: "When you click the "New Slide" button, it makes a new slide for you"

> Brother: "What if I click it again?"

> You: "It makes another"

> Brother: "And AGAIN?"

Your brother keeps spamming the "New Slide" button. Suddenly, he notices that the slide number $10$ has reached double digits. He hasn't yet learnt the concept of place value, so he finds this bizarre. You teach him that $10$ is the number after $9$. This blows his mind.

> Brother: So there are $2$ numbers inside a bigger number?

> You: Yes, they're called ***digits***

> Brother: Hmmmm, so how many ***digits*** are there in total in the Orange Word?

> You: Among the slide numbers? Well, the first $9$ numbers all have $1$ digit each. And $10$ has $2$ digits. So in total, there's $11$ digits.

Your brother continues spamming until it reaches $100$

> Brother: Now there are $3$ numbers???

> You: Yep, you've discovered the triple digits

> Brother: Oh. Then how many digits in total now?

This question stumps you. Your arithmetic skills are abysmal, so you tell him to wait as you write up an algorithm.

## Brute Force Approach - $O(n)$

In [14]:
import numpy as np

digits = 0
total = 0
for i in range(1, 100+1):
    if np.log10(i) % 1 == 0:     # reached a power of 10
        digits += 1
    total += digits

print("Answer:", total)

Answer: 192


In $30$ s flat, you write up this code and triumphantly announce the answer to be $192$ digits.

Your brother has not faltered in his spamming of new slides. He now asks the same question repeatedly, regarding $539$, $540$ and $541$ slides. You decide to generalise your algorithm so your brother can ask the interface directly instead of through the intermediary of you.

In [4]:

import numpy as np
# O(n) solution, where n is the number of slides

def bruteforce(n):
    digits = 0
    total = 0
    for i in range(1, n+1):
        if np.log10(i) % 1 == 0:     # reached a power of 10
            digits += 1
        total += digits

    return total

n = int(input("Largest Slide Number:"))
print(f"Total Number of Digits: {bruteforce(n)}")

Total Number of Digits: 2893


Your brother is happy now.

## Iterative Approach - $O(\log n)$

But that's not enough. It doesn't matter that you're brother is happy - *you*'re not happy. The bruteforce solution was clunky and inefficient. As an aspiring computer scientist, you strive for a better algorithm.

The most obvious flaw is that the for loop is just executing repeated addition, unnecessarily impeding the time complexity. A good optimisation is to instead calculate how many slide numbers have a certain number of digits, then multiply that quantity by said number of digits.

---

You consider $n=105$

There are $9$ single digit numbers: $(1, 2, \cdots, 8, 9)$<br>
Digits: $9\cdot1=9$


There are then $99-9=90$ double digit numbers: $(10, 11, \cdots, 98, 99)$<br>
Digits: $90\cdot2=180$

And finally there are $105-99=6$ triple digit numbers left over: $(100, 101, \cdots, 104, 105)$<br>
Digits: $6\cdot3=18$

Therefore the total number of digits is $9+180+18=207$

---

To generalise your thinking, you first want to quantify the number of digits of a number $n$. Due to our base $10$ number system, $10^l$ has $l+1$ digits $\forall l\in\mathbb{N}_0$. Any $n\in\mathbb{N}$ can be expressed in scientific notation as $a\cdot10^l, a\in[1,10)$. $l$ is therefore $\lfloor \log_{10}(n)\rfloor$, and the number of digits of $n$ is $d = l+1$.

You then recognise that there exist $10^d - 10^{d-1} = 9(10^{d-1})$ $d$-digit natural numbers. So if your max slide number is $n$, you can first consider all natural numbers with no more than $d-1=l$ digits by evaluating $\sum_{i=0}^l 9(10^i)\text{ numbers }\cdot (i+1)\text{ digits }$.

You realise that there will be a remaining surplus of $n - (10^l - 1) = n - 10^l + 1$ numbers, each with $d=l+1$ digits, that must be summed separately.

You collect your findings into an algorithm. The sum is evaluated iteratively, so the time complexity is $O(\log n)$.

In [6]:
import numpy as np
# O(logn) solution, where n is the number of slides

n = int(input("number of slides: "))
exp = int(np.floor(np.log10(n)))        # exponent of 10 i.e. number of digits - 1
total = 0

# sum up to highest power of 10 in O(logn) time
for smaller_exp in range(exp):
  smaller_n = 9 * 10**smaller_exp
  smaller_digits = smaller_exp + 1
  total += smaller_digits * smaller_n

# sum surplus in O(1) time
surplus = n - 10**exp + 1
digits = exp + 1
total += digits * surplus 

print("digits:", total)

digits: 2893


## Series Evaluation Approach - $O(1)$

Your optimisation was good, but you think you can do better. You recall geometric partial sums $\sum_{i=0}^n a\cdot r^i = a\cdot\frac{r^{n+1} - 1}{r-1}$ from your recently full-marked spesh paper, and note that the iteratively computed sum $\sum_{i=0}^l 9(10^i)(i+1)$ is eerily similar.

That's when you're hit with a eureka moment. $\frac{d}{dx} r^i = (i)r^{i-1} \implies \frac{d}{dx} r^{i+1} = (i+1)r^i$, and derivatives carry through summations by the property $\frac{d}{dx} (f(x) + g(x)) \equiv \frac{d}{dx} f(x) + \frac{d}{dx} g(x)$. Therefore,

\begin{align*}

\sum_{i=0}^n r^{i+1} &= \sum_{i=0}^n r(r^i)\\\\

&= r\left( \frac{r^{n+1}-1}{r-1} \right)\quad \because\text{geometric partial sum}\\\\

&= \left( 1+\frac{1}{r-1} \right)    \left( r^{n+1}-1 \right)\\\\

\sum_{i=0}^n (i+1)r^i &= \left(\frac{-1}{(r-1)^2}\right)(r^{n+1}-1)   +   \left(1+\frac{1}{r-1}\right)(n+1)r^n\\\\

&= \frac{1-r^{n+1} + (r-1)r(n+1)r^n}{(r-1)^2}\\\\

&= \frac{1-r^{n+1} + r^{n+2}(n+1) - r^{n+1}(n+1)}{(r-1)^2}\\\\

&= \frac{1 + (n+1)r^{n+2} - (n+2)r^{n+1}}    {(r-1)^2}\\\\\\




\sum_{i=0}^{l-1} 9(10^i)(i+1) &= 9\sum_{i=0}^{l-1} (i+1)10^i\\\\

&= 9\cdot\frac{1 + (l)10^{l+1} - (l+1)10^{l}}    {(10-1)^2}\\\\

&= \frac{1 + l(10)^{l+1} - (l+1)(10)^{l}}    {9}\\

\end{align*}


The calculation of the surplus $(l+1)(n-10^l+1)$ is already $O(1)$ optimised.

Therefore, by summing these two quantities, an $O(1)$ algorithm for the total number of digits can be obtained.

In [2]:
import numpy as np
# O(1) solution

n = int(input("number of slides: "))
exp = int(np.floor(np.log10(n)))

power10 = (1 + exp*10**(exp+1) - (exp+1)*10**exp) // 9
surplus = (exp+1) * (n - 10**exp + 1)
total = power10 + surplus

print("total number of digits:", total)

total number of digits: 5888896
